## IMPORTS

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

import os
import matplotlib.pyplot as plt

import sklearn
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from scipy.stats import zscore, pearsonr, uniform
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, StratifiedKFold, RandomizedSearchCV
from sklearn.impute import KNNImputer
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.utils import resample

from scipy.io import loadmat

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer, accuracy_score, f1_score

In [3]:
# file_prefix = "/kaggle/input/widsdatathon2025/"
file_prefix = "data/"

## LOAD CAT DATA

In [4]:
file_path_trainC = file_prefix + "TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx"
train_cat = pd.read_excel(file_path_trainC)
print(train_cat.shape)
train_cat.head()

(1213, 10)


,participant_id,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ
0,UmrK0vMLopoR,2016,1,0.0,0,1,21,45,21,45
1,CPaeQkhcjg7d,2019,3,1.0,2,3,15,15,0,0
2,Nb4EetVPm3gs,2016,1,1.0,8,1,18,40,0,0
3,p4vPhVu91o4b,2018,3,0.0,8,3,15,30,18,0
4,M09PXs7arQ5E,2019,3,0.0,1,3,15,20,0,0


In [5]:
file_path_testC = file_prefix + "TEST/TEST_CATEGORICAL.xlsx"
test_cat = pd.read_excel(file_path_testC)
test_cat.head()

,participant_id,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ
0,Cfwaf5FX7jWK,2022,4,0.0,0.0,4,21.0,30.0,18.0,30.0
1,vhGrzmvA3Hjq,2023,4,0.0,0.0,4,21.0,45.0,NaN,30.0
2,ULliyEXjy4OV,2022,4,0.0,0.0,4,21.0,40.0,18.0,40.0
3,LZfeAb1xMtql,2022,4,0.0,0.0,3,21.0,45.0,21.0,45.0
4,EnFOUv0YK1RG,2022,4,2.0,0.0,4,18.0,0.0,21.0,45.0


In [6]:
file_path_trainS = file_prefix + "TRAIN/TRAINING_SOLUTIONS.xlsx"
train_Solutions = pd.read_excel(file_path_trainS)
print(train_Solutions.shape)
train_Solutions.head()

(1213, 3)


,participant_id,ADHD_Outcome,Sex_F
0,UmrK0vMLopoR,1,1
1,CPaeQkhcjg7d,1,0
2,Nb4EetVPm3gs,1,0
3,p4vPhVu91o4b,1,1
4,M09PXs7arQ5E,1,1


## PREPROCESS CAT DATA: 
`cat_train_final` and `cat_test_final`

In [7]:
train_cat = train_cat.drop(columns=['Basic_Demos_Enroll_Year', 'Basic_Demos_Study_Site', 'MRI_Track_Scan_Location'])
for col in train_cat.select_dtypes(include='int').columns:
    train_cat[col] = train_cat[col].astype('category')
# Creating a list of all of the columns except the first
columns_to_encode = train_cat.columns[1:].tolist()

# Print the columns to encode
print("Columns to encode:", columns_to_encode)
# encoding categorical data
train_encoded = pd.get_dummies(train_cat[columns_to_encode], dummy_na=True, drop_first=True)
train_encoded = train_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))

ethnicity_one_hot = pd.get_dummies(train_cat['PreInt_Demos_Fam_Child_Ethnicity'], prefix="PreInt_Demos_Fam_Child_Ethnicity", dummy_na=True)
ethnicity_one_hot = ethnicity_one_hot.applymap(lambda x: 1 if x is True else (0 if x is False else x))
ethnicity_one_hot = ethnicity_one_hot.rename(columns=lambda x: x.rstrip('.0'))
ethnicity_one_hot = ethnicity_one_hot.rename(columns={"PreInt_Demos_Fam_Child_Ethnicity_": "PreInt_Demos_Fam_Child_Ethnicity_0"})

cat_train_final = pd.concat([train_cat.drop(columns=columns_to_encode), train_encoded], axis=1)
cat_train_final = pd.concat([cat_train_final, ethnicity_one_hot], axis=1)
cat_train_final.drop(columns=['PreInt_Demos_Fam_Child_Ethnicity'], inplace=True)

Columns to encode: ['PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ']


/var/folders/qf/nd5hbd952q710b0vwz8x7q_40000gn/T/ipykernel_68717/1115194704.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_encoded = train_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))
/var/folders/qf/nd5hbd952q710b0vwz8x7q_40000gn/T/ipykernel_68717/1115194704.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ethnicity_one_hot = ethnicity_one_hot.applymap(lambda x: 1 if x is True else (0 if x is False else x))


In [8]:
test_cat = test_cat.drop(columns=['Basic_Demos_Enroll_Year', 'Basic_Demos_Study_Site', 'MRI_Track_Scan_Location'])
# convert our int variables to categories
for col in test_cat.select_dtypes(include='float').columns:
    test_cat[col] = test_cat[col].astype('category')
# Encode categorical variables in test
test_encoded = pd.get_dummies(test_cat[columns_to_encode], dummy_na=True, drop_first=True)
test_encoded = test_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))
test_encoded = test_encoded.rename(columns=lambda x: x.rstrip('.0'))
cat_train_final_cols = cat_train_final.columns.tolist()
cat_train_final_cols.remove('participant_id')

missing_cols = set(cat_train_final_cols) - set(test_encoded.columns)
print(len(missing_cols), "MISSING COLS")
print(missing_cols)
for col in missing_cols:
    if col in test_encoded.columns:
        print("COL IN TEST ENCODED")
        print(col)
    else:
        test_encoded[col] = 0
# Ensure test_encoded columns are in the same order as train_encoded
test_encoded = test_encoded.reindex(columns=cat_train_final_cols, fill_value=0)

# Combine encoded columns with the rest of the DataFrame
cat_test_final = pd.concat([test_cat.drop(columns=columns_to_encode), test_encoded], axis=1)

12 MISSING COLS
{'Barratt_Barratt_P1_Occ_20', 'Barratt_Barratt_P1_Edu_3', 'Barratt_Barratt_P2_Edu_3', 'Barratt_Barratt_P2_Occ_20', 'Barratt_Barratt_P2_Occ_30', 'Barratt_Barratt_P1_Occ_40', 'Barratt_Barratt_P2_Occ_40', 'PreInt_Demos_Fam_Child_Race_10', 'Barratt_Barratt_P1_Occ_30', 'Barratt_Barratt_P1_Occ_10', 'PreInt_Demos_Fam_Child_Ethnicity_0', 'Barratt_Barratt_P2_Occ_10'}


/var/folders/qf/nd5hbd952q710b0vwz8x7q_40000gn/T/ipykernel_68717/1798571501.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_encoded = test_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))


In [9]:
cat_test_final

,participant_id,PreInt_Demos_Fam_Child_Race_1,PreInt_Demos_Fam_Child_Race_2,PreInt_Demos_Fam_Child_Race_3,PreInt_Demos_Fam_Child_Race_4,PreInt_Demos_Fam_Child_Race_7,PreInt_Demos_Fam_Child_Race_8,PreInt_Demos_Fam_Child_Race_9,PreInt_Demos_Fam_Child_Race_10,PreInt_Demos_Fam_Child_Race_11,...,Barratt_Barratt_P2_Occ_30,Barratt_Barratt_P2_Occ_35,Barratt_Barratt_P2_Occ_40,Barratt_Barratt_P2_Occ_45,Barratt_Barratt_P2_Occ_nan,PreInt_Demos_Fam_Child_Ethnicity_0,PreInt_Demos_Fam_Child_Ethnicity_1,PreInt_Demos_Fam_Child_Ethnicity_2,PreInt_Demos_Fam_Child_Ethnicity_3,PreInt_Demos_Fam_Child_Ethnicity_nan
0,Cfwaf5FX7jWK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,vhGrzmvA3Hjq,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ULliyEXjy4OV,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,LZfeAb1xMtql,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,EnFOUv0YK1RG,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,UadZfjdEg7eG,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
300,IUEHiLmQAqCi,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
301,cRySmCadYFRO,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
302,E3MvDUtJadc5,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [10]:
cat_train_final

,participant_id,PreInt_Demos_Fam_Child_Race_1,PreInt_Demos_Fam_Child_Race_2,PreInt_Demos_Fam_Child_Race_3,PreInt_Demos_Fam_Child_Race_4,PreInt_Demos_Fam_Child_Race_7,PreInt_Demos_Fam_Child_Race_8,PreInt_Demos_Fam_Child_Race_9,PreInt_Demos_Fam_Child_Race_10,PreInt_Demos_Fam_Child_Race_11,...,Barratt_Barratt_P2_Occ_30,Barratt_Barratt_P2_Occ_35,Barratt_Barratt_P2_Occ_40,Barratt_Barratt_P2_Occ_45,Barratt_Barratt_P2_Occ_nan,PreInt_Demos_Fam_Child_Ethnicity_0,PreInt_Demos_Fam_Child_Ethnicity_1,PreInt_Demos_Fam_Child_Ethnicity_2,PreInt_Demos_Fam_Child_Ethnicity_3,PreInt_Demos_Fam_Child_Ethnicity_nan
0,UmrK0vMLopoR,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,CPaeQkhcjg7d,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,Nb4EetVPm3gs,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,p4vPhVu91o4b,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,M09PXs7arQ5E,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,Atx7oub96GXS,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1209,groSbUfkQngM,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1210,zmxGvIrOD0bt,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1211,rOmWFuJCud5G,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0


## LOAD QUANT DATA

In [11]:
file_path_trainQ = file_prefix + "TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx"
file_path_testQ = file_prefix + "TEST/TEST_QUANTITATIVE_METADATA.xlsx"
quant_train_df = pd.read_excel(file_path_trainQ)
quant_test_df = pd.read_excel(file_path_testQ)

## PREPROCESS QUANT DATA:
`quant_train_df` and `quant_test_df`

In [12]:
quant_train_df.fillna({'MRI_Track_Age_at_Scan':quant_train_df['MRI_Track_Age_at_Scan'].median()}, inplace = True)

In [13]:
knn_columns = quant_test_df.columns[quant_test_df.isna().any()].tolist()

imputer = KNNImputer(n_neighbors=5)

# Apply imputation to selected columns
quant_test_df[knn_columns] = imputer.fit_transform(quant_test_df[knn_columns])

# Verify missing values are handled
print(quant_test_df.isnull().sum())

participant_id                0
EHQ_EHQ_Total                 0
ColorVision_CV_Score          0
APQ_P_APQ_P_CP                0
APQ_P_APQ_P_ID                0
APQ_P_APQ_P_INV               0
APQ_P_APQ_P_OPD               0
APQ_P_APQ_P_PM                0
APQ_P_APQ_P_PP                0
SDQ_SDQ_Conduct_Problems      0
SDQ_SDQ_Difficulties_Total    0
SDQ_SDQ_Emotional_Problems    0
SDQ_SDQ_Externalizing         0
SDQ_SDQ_Generating_Impact     0
SDQ_SDQ_Hyperactivity         0
SDQ_SDQ_Internalizing         0
SDQ_SDQ_Peer_Problems         0
SDQ_SDQ_Prosocial             0
MRI_Track_Age_at_Scan         0
dtype: int64


## LOAD FCM DATA AND SOLUTIONS

In [14]:
train_fcm_df = pd.read_csv(file_prefix + 'TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv')
test_fcm_df = pd.read_csv(file_prefix + 'TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')

train_Solutions = pd.read_excel(file_prefix + 'TRAIN/TRAINING_SOLUTIONS.xlsx')

## MERGE DATASETS

In [15]:
train_cat_FCM = pd.merge(cat_train_final, train_fcm_df, on = 'participant_id')
train_df = pd.merge(train_cat_FCM, quant_train_df, on = 'participant_id')
train_df.head()

,participant_id,PreInt_Demos_Fam_Child_Race_1,PreInt_Demos_Fam_Child_Race_2,PreInt_Demos_Fam_Child_Race_3,PreInt_Demos_Fam_Child_Race_4,PreInt_Demos_Fam_Child_Race_7,PreInt_Demos_Fam_Child_Race_8,PreInt_Demos_Fam_Child_Race_9,PreInt_Demos_Fam_Child_Race_10,PreInt_Demos_Fam_Child_Race_11,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,UmrK0vMLopoR,0,0,0,0,0,0,0,0,0,...,0,6,1,5,0,5,1,0,10,10.739219
1,CPaeQkhcjg7d,0,1,0,0,0,0,0,0,0,...,0,18,6,8,7,8,10,4,5,10.739219
2,Nb4EetVPm3gs,0,0,0,0,0,1,0,0,0,...,1,14,2,8,5,7,6,4,9,8.239904
3,p4vPhVu91o4b,0,0,0,0,0,1,0,0,0,...,6,24,4,16,9,10,8,4,6,10.739219
4,M09PXs7arQ5E,1,0,0,0,0,0,0,0,0,...,1,18,4,11,4,10,7,3,9,8.940679


In [16]:
test_cat_FCM = pd.merge(cat_test_final, test_fcm_df, on = 'participant_id')
test_df = pd.merge(test_cat_FCM, quant_test_df, on = 'participant_id')
test_df.head()

,participant_id,PreInt_Demos_Fam_Child_Race_1,PreInt_Demos_Fam_Child_Race_2,PreInt_Demos_Fam_Child_Race_3,PreInt_Demos_Fam_Child_Race_4,PreInt_Demos_Fam_Child_Race_7,PreInt_Demos_Fam_Child_Race_8,PreInt_Demos_Fam_Child_Race_9,PreInt_Demos_Fam_Child_Race_10,PreInt_Demos_Fam_Child_Race_11,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,Cfwaf5FX7jWK,0,0,0,0,0,0,0,0,0,...,2.0,12.0,3.0,9.0,2.0,7.0,3.0,0.0,8.0,8.992813
1,vhGrzmvA3Hjq,0,0,0,0,0,0,0,0,0,...,2.0,16.0,8.0,5.0,7.0,3.0,11.0,3.0,9.0,12.324093
2,ULliyEXjy4OV,0,0,0,0,0,0,0,0,0,...,1.0,7.0,1.0,6.0,1.0,5.0,1.0,0.0,9.0,7.770933
3,LZfeAb1xMtql,0,0,0,0,0,0,0,0,0,...,4.0,15.0,4.0,10.0,8.0,6.0,5.0,1.0,6.0,9.304814
4,EnFOUv0YK1RG,0,0,0,0,0,0,0,0,0,...,2.0,18.0,6.0,12.0,5.0,10.0,6.0,0.0,10.0,8.261350


## Undersample to address class imbalance

In [138]:
# merge train_df with train_Solutions
total_df = pd.merge(train_df, train_Solutions, on = 'participant_id')
total_df.head()

,participant_id,PreInt_Demos_Fam_Child_Race_1,PreInt_Demos_Fam_Child_Race_2,PreInt_Demos_Fam_Child_Race_3,PreInt_Demos_Fam_Child_Race_4,PreInt_Demos_Fam_Child_Race_7,PreInt_Demos_Fam_Child_Race_8,PreInt_Demos_Fam_Child_Race_9,PreInt_Demos_Fam_Child_Race_10,PreInt_Demos_Fam_Child_Race_11,...,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan,ADHD_Outcome,Sex_F
0,UmrK0vMLopoR,0,0,0,0,0,0,0,0,0,...,1,5,0,5,1,0,10,10.739219,1,1
1,CPaeQkhcjg7d,0,1,0,0,0,0,0,0,0,...,6,8,7,8,10,4,5,10.739219,1,0
2,Nb4EetVPm3gs,0,0,0,0,0,1,0,0,0,...,2,8,5,7,6,4,9,8.239904,1,0
3,p4vPhVu91o4b,0,0,0,0,0,1,0,0,0,...,4,16,9,10,8,4,6,10.739219,1,1
4,M09PXs7arQ5E,1,0,0,0,0,0,0,0,0,...,4,11,4,10,7,3,9,8.940679,1,1


In [139]:
# Separate majority and minority classes
adhd_majority = total_df[total_df["ADHD_Outcome"] == 1]
adhd_minority = total_df[total_df["ADHD_Outcome"] == 0]

sex_majority = total_df[total_df["Sex_F"] == 0]
sex_minority = total_df[total_df["Sex_F"] == 1]

# Downsample ADHD_Outcome=1 to match ADHD_Outcome=0
adhd_majority_downsampled = resample(adhd_majority, 
                                     replace=False,    # Sample without replacement
                                     n_samples=len(adhd_minority), 
                                     random_state=42)

# Downsample Sex_F=0 to match Sex_F=1
sex_majority_downsampled = resample(sex_majority, 
                                    replace=False, 
                                    n_samples=len(sex_minority), 
                                    random_state=42)

# Combine downsampled groups with the minority classes
df_balanced = pd.concat([adhd_majority_downsampled, adhd_minority, sex_majority_downsampled, sex_minority])

# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


In [140]:
df_balanced['ADHD_Outcome'].value_counts()

ADHD_Outcome
1    935
0    661
Name: count, dtype: int64

In [141]:
df_balanced['Sex_F'].value_counts()

Sex_F
0    903
1    693
Name: count, dtype: int64

In [17]:
X_train = train_df.drop(columns = ['participant_id'])
Y_train = train_Solutions.drop(columns = ['participant_id'])
participant_id = test_df['participant_id']
X_test = test_df.drop(columns = 'participant_id')
# X_train = total_df.drop(columns = ['participant_id', 'ADHD_Outcome', 'Sex_F'])
# Y_train = total_df[['ADHD_Outcome', 'Sex_F']]
# participant_id = test_df['participant_id']
# X_test = test_df.drop(columns = 'participant_id')

In [18]:
Y_train

,ADHD_Outcome,Sex_F
0,1,1
1,1,0
2,1,0
3,1,1
4,1,1
...,...,...
1208,0,0
1209,0,1
1210,0,1
1211,0,0


In [157]:
X_test

,PreInt_Demos_Fam_Child_Race_1,PreInt_Demos_Fam_Child_Race_2,PreInt_Demos_Fam_Child_Race_3,PreInt_Demos_Fam_Child_Race_4,PreInt_Demos_Fam_Child_Race_7,PreInt_Demos_Fam_Child_Race_8,PreInt_Demos_Fam_Child_Race_9,PreInt_Demos_Fam_Child_Race_10,PreInt_Demos_Fam_Child_Race_11,PreInt_Demos_Fam_Child_Race_nan,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,0,0,0,0,0,0,0,0,0,0,...,2.0,12.0,3.0,9.0,2.0,7.0,3.0,0.0,8.0,8.992813
1,0,0,0,0,0,0,0,0,0,0,...,2.0,16.0,8.0,5.0,7.0,3.0,11.0,3.0,9.0,12.324093
2,0,0,0,0,0,0,0,0,0,0,...,1.0,7.0,1.0,6.0,1.0,5.0,1.0,0.0,9.0,7.770933
3,0,0,0,0,0,0,0,0,0,0,...,4.0,15.0,4.0,10.0,8.0,6.0,5.0,1.0,6.0,9.304814
4,0,0,0,0,0,0,0,0,0,0,...,2.0,18.0,6.0,12.0,5.0,10.0,6.0,0.0,10.0,8.261350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,0,0,0,0,0,0,0,0,0,0,...,1.0,18.0,7.0,7.0,5.0,6.0,11.0,4.0,7.0,7.546999
300,1,0,0,0,0,0,0,0,0,0,...,2.0,16.0,2.0,11.0,5.0,9.0,5.0,3.0,8.0,10.531143
301,0,0,0,0,0,1,0,0,0,0,...,1.0,11.0,4.0,4.0,4.0,3.0,7.0,3.0,10.0,7.210586
302,0,0,0,0,0,1,0,0,0,0,...,5.0,21.0,2.0,10.0,6.0,5.0,11.0,9.0,0.0,12.212183


## FEATURE SELECTION WITH LOGISTIC REGRESSION

In [19]:
model = LogisticRegression(penalty='l1', solver='liblinear')
model.fit(train_df.drop(columns='participant_id'), train_Solutions['Sex_F'])
selected_features_Sex = train_df.drop(columns='participant_id').columns[model.coef_[0] != 0]
print(selected_features_Sex)

Index(['PreInt_Demos_Fam_Child_Race_1', 'PreInt_Demos_Fam_Child_Race_9',
       'Barratt_Barratt_P1_Edu_9', 'Barratt_Barratt_P1_Edu_18',
       'Barratt_Barratt_P1_Edu_21', 'Barratt_Barratt_P1_Occ_5',
       'Barratt_Barratt_P1_Occ_20', 'Barratt_Barratt_P1_Occ_25',
       'Barratt_Barratt_P1_Occ_35', 'Barratt_Barratt_P1_Occ_40',
       ...
       'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP',
       'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing',
       'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
       'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan'],
      dtype='object', length=390)


In [20]:
model = LogisticRegression(penalty='l1', solver='liblinear')
model.fit(train_df.drop(columns='participant_id'), train_Solutions['ADHD_Outcome'])
selected_features_ADHD = train_df.drop(columns='participant_id').columns[model.coef_[0] != 0]
print(selected_features_ADHD)

Index(['PreInt_Demos_Fam_Child_Race_1', 'PreInt_Demos_Fam_Child_Race_3',
       'PreInt_Demos_Fam_Child_Race_8', 'Barratt_Barratt_P1_Edu_9',
       'Barratt_Barratt_P1_Edu_12', 'Barratt_Barratt_P1_Edu_18',
       'Barratt_Barratt_P1_Edu_21', 'Barratt_Barratt_P1_Occ_5',
       'Barratt_Barratt_P1_Occ_20', 'Barratt_Barratt_P1_Occ_25',
       ...
       'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems',
       'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing',
       'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
       'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan'],
      dtype='object', length=278)


In [ ]:
# get the top 10 features for each model
top_features_ADHD = list(set(selected_features_ADHD
                        .sort_values(ascending=False)[:10]
                        .index)
                    .union(set(selected_features_ADHD
                               .sort_values(ascending=False)[:10]
                               .index)))
top_features_Sex = list(set
                        (selected_features_Sex
                        .sort_values(ascending=False)[:10]
                        .index)
                    .union(set(selected_features_Sex
                               .sort_values(ascending=False)[:10]
                               .index)))    
print(top_features)

In [21]:
common_features = list(set(selected_features_ADHD) & set(selected_features_Sex))
X_train_2 = X_train[common_features]
X_test_2 = X_test[common_features]

## MODELING

In [22]:
Y_train['ADHD_Outcome']

0       1
1       1
2       1
3       1
4       1
       ..
1208    0
1209    0
1210    0
1211    0
1212    0
Name: ADHD_Outcome, Length: 1213, dtype: int64

In [23]:
Y_train['Sex_F']

0       1
1       0
2       0
3       1
4       1
       ..
1208    0
1209    1
1210    1
1211    0
1212    0
Name: Sex_F, Length: 1213, dtype: int64

## 2 Separate Models

In [24]:
# grid cv search for hyperparametersn XGBClassifier(objective='binary:logistic', n_estimators=100, learning_rate=0.1, max_depth=5)
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [2, 3, 4, 5, 8]
}

xgb_classifier_ADHD = XGBClassifier(objective='binary:logistic')
xgb_classifier_sex = XGBClassifier(objective='binary:logistic')

grid_search_ADHD = GridSearchCV(
    estimator=xgb_classifier_ADHD,
    param_grid=param_grid,
    scoring='f1_weighted', 
    cv=5, 
    verbose=1,
)

grid_search_sex = GridSearchCV(
    estimator=xgb_classifier_sex,
    param_grid=param_grid,
    scoring='f1_weighted', 
    cv=5, 
    verbose=1,
)

grid_search_ADHD.fit(X_train_2, Y_train['ADHD_Outcome'])
grid_search_sex.fit(X_train_2, Y_train['Sex_F'])

Fitting 5 folds for each of 45 candidates, totalling 225 fits
Fitting 5 folds for each of 45 candidates, totalling 225 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [2, 3, 4, 5, 8],
                         'n_estimators': [50, 100, 200]},
             scoring='f1_weighted', verbose=1)

In [25]:
print("Best Parameters:", grid_search_ADHD.best_params_)
print("Best F1 Score:", grid_search_ADHD.best_score_)

# Use the best model
best_model_ADHD = grid_search_ADHD.best_estimator_

Best Parameters: {'learning_rate': 0.2, 'max_depth': 2, 'n_estimators': 50}
Best F1 Score: 0.7869571169014055


In [26]:
print("Best Parameters:", grid_search_sex.best_params_)
print("Best F1 Score:", grid_search_sex.best_score_)

# Use the best model
best_model_sex = grid_search_sex.best_estimator_


Best Parameters: {'learning_rate': 0.2, 'max_depth': 8, 'n_estimators': 100}
Best F1 Score: 0.6260380402285701


In [29]:
# create new xgbclassifier with best parameters
final_xgb_ADHD = XGBClassifier(objective='binary:logistic', n_estimators=50, learning_rate=0.2, max_depth=2)
final_xgb_sex = XGBClassifier(objective='binary:logistic', n_estimators=100, learning_rate=0.2, max_depth=8)

final_xgb_ADHD.fit(X_train_2, Y_train['ADHD_Outcome'])
final_xgb_sex.fit(X_train_2, Y_train['Sex_F'])

print("ADHD f1 score", f1_score(Y_train['ADHD_Outcome'], final_xgb_ADHD.predict(X_train_2)))
print("sex f1 score", f1_score(Y_train['Sex_F'], final_xgb_sex.predict(X_train_2)))

y_pred_ADHD = final_xgb_ADHD.predict(X_test_2)
y_pred_sex = final_xgb_sex.predict(X_test_2)

ADHD f1 score 0.8944954128440367
sex f1 score 1.0


In [31]:
y_pred_ADHD.shape

(304,)

In [32]:
y_pred_sex.shape

(304,)

In [33]:
print(type(y_pred_ADHD))

<class 'numpy.ndarray'>


In [40]:
# create dataframe with participant_id, ADHD_Ooutcome, and Sex_F
predictions_df = pd.DataFrame({
    'ADHD_Outcome': y_pred_ADHD,
    'Sex_F': y_pred_sex
}
)

# Combine participant IDs with predictions
result_df_2 = pd.concat([participant_id.reset_index(drop=True), predictions_df], axis=1)
result_df_2.head()

,participant_id,ADHD_Outcome,Sex_F
0,Cfwaf5FX7jWK,1,0
1,vhGrzmvA3Hjq,1,0
2,ULliyEXjy4OV,1,0
3,LZfeAb1xMtql,1,0
4,EnFOUv0YK1RG,1,0


In [41]:
# save to csv file
result_df_2.to_csv('submission.csv', index=False)

In [ ]:
y_pred_ADHD = best_model_ADHD.predict(X_test_2)
y_pred_sex = best_model_sex.predict(X_test_2)

## 1 Model

In [42]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [2, 3, 4, 5, 8]
}

xgb_classifier = XGBClassifier(objective='binary:logistic')

grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    scoring='f1_weighted', 
    cv=5, 
    verbose=1,
)

grid_search.fit(X_train_2, Y_train)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [2, 3, 4, 5, 8],
                         'n_estimators': [50, 100, 200]},
             scoring='f1_weighted', verbose=1)

In [43]:
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

# Use the best model
best_model = grid_search.best_estimator_

Best Parameters: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 200}
Best F1 Score: 0.6551995857467163


In [44]:
y_pred = best_model.predict(X_train_2)
print("Accuracy:", accuracy_score(Y_train, y_pred))
print("F1 score", f1_score(Y_train, y_pred, average='weighted'))

Accuracy: 1.0
F1 score 1.0


In [45]:
xgb_classifier = XGBClassifier(objective='binary:logistic', n_estimators=200, learning_rate=0.2, max_depth=5)
multioutput_classifier = MultiOutputClassifier(xgb_classifier)
multioutput_classifier.fit(X_train_2, Y_train)
y_pred = multioutput_classifier.predict(X_test_2)

In [46]:
y_pred

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [1, 1],
       [0, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 1],
       [1, 0],
       [1, 1],
       [1, 0],
       [1, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1,

In [47]:
predictions_df_2 = pd.DataFrame(
    y_pred,
    columns=['ADHD_Outcome', 'Sex_F']
)

# Combine participant IDs with predictions
result_df_2 = pd.concat([participant_id.reset_index(drop=True), predictions_df_2], axis=1)
result_df_2.head()

,participant_id,ADHD_Outcome,Sex_F
0,Cfwaf5FX7jWK,1,0
1,vhGrzmvA3Hjq,1,0
2,ULliyEXjy4OV,1,0
3,LZfeAb1xMtql,1,0
4,EnFOUv0YK1RG,1,0


In [ ]:
result_df_2.to_csv('submission.csv', index=False)


In [152]:
def multi_output_accuracy(y_true, y_pred):
    # Ensure y_true and y_pred are NumPy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    # Compute accuracy for each target variable and return the mean
    return np.mean([accuracy_score(y_true[:, i], y_pred[:, i]) for i in range(y_true.shape[1])])
multi_output_scorer = make_scorer(multi_output_accuracy)

In [153]:
cv_scores_2 = cross_val_score(multioutput_classifier, X_train_2, Y_train, cv=5, scoring=multi_output_scorer)

# Output the cross-validation results
print("Cross-validation scores for each fold:", cv_scores_2)
print("Mean CV score:", np.mean(cv_scores_2))

Cross-validation scores for each fold: [0.79012346 0.77160494 0.70781893 0.64049587 0.44008264]
Mean CV score: 0.6700251674999149


In [48]:
f1_score(Y_train, multioutput_classifier.predict(X_train_2), average='weighted') # Weighs F1 scores based on the number of occurrences of each label (helps if classes are imbalanced)

1.0

In [155]:
# save result_df_2 to csv
result_df_2.to_csv(file_prefix + 'submission.csv', index=False)